In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
video_L_path = os.path.join(DATA_FOLDER, "keparoicam_clipL_synchronized.mp4")
video_R_path = os.path.join(DATA_FOLDER, "keparoicam_clipR_synchronized.mp4")
video_left_capture = cv2.VideoCapture(video_L_path)
video_right_capture = cv2.VideoCapture(video_R_path)

In [ ]:
left_test_image = cv2.imread(os.path.join(DATA_FOLDER, "keparoi_left_frame.jpg"))
right_test_image = cv2.imread(os.path.join(DATA_FOLDER, "keparoi_right_frame.jpg"))

In [ ]:
def prepare_frame(frame, mask):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.GaussianBlur(frame, (9,9), 0)
    frame = cv2.bitwise_and(frame, frame, mask=mask)
    return frame

In [ ]:
def find_absolute_diff_frame(frame1, frame2):
    diff_frame = cv2.absdiff(frame1, frame2)

    kernel = np.ones((5, 5))
    diff_frame = cv2.dilate(diff_frame, kernel, 1)
    
    thresh_frame = cv2.threshold(src=diff_frame, thresh=30, maxval=255, type=cv2.THRESH_BINARY)[1]
    return thresh_frame

In [ ]:
def mask_field_from_image(frame):
    median_blurred_image = cv2.medianBlur(frame, 51)
    gray_image = cv2.cvtColor(median_blurred_image, cv2.COLOR_RGB2GRAY)
    _ , mask = cv2.threshold(gray_image, 100, 255, cv2.THRESH_BINARY)
    mask = ~mask
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (151,151))
    mask = cv2.morphologyEx(mask,cv2.MORPH_CLOSE, kernel)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    max_contour = max(contours, key = cv2.contourArea)
    approx = cv2.approxPolyDP(max_contour, 0.1 * cv2.arcLength(max_contour, True), True)
    final_mask = np.zeros(gray_image.shape, np.uint8)
    cv2.drawContours(final_mask, [approx], 0, (255),-1)
    final_mask = ~final_mask
    return final_mask

In [ ]:
ret_left, frame_left = video_left_capture.read()
ret_right, frame_right = video_right_capture.read()

left_mask = mask_field_from_image(frame_left)
right_mask = mask_field_from_image(frame_right)

plt.imshow(left_mask)
plt.show()
plt.imshow(right_mask)
plt.show()

In [ ]:
prev_left = None
prev_right = None

for i in range(100):
    
    frame_number = i*10
    
    video_left_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    video_right_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    
    res_left, frame_left = video_left_capture.read()
    res_right, frame_right = video_right_capture.read()
    
    if i == 0:
        prev_left = frame_left
        prev_right = frame_right
        continue
    
    thresh_l = find_absolute_diff_frame(prev_left, frame_left)
    thresh_r = find_absolute_diff_frame(prev_right, frame_right)

    contours_l, _ = cv2.findContours(image=thresh_l, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    contours_r, _ = cv2.findContours(image=thresh_r, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)

    
    #left_copy = frame_left.copy()
    #right_copy = frame_right.copy()
    #cv2.drawContours(image=left_copy, contours=contours_l, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
    #cv2.drawContours(image=right_copy, contours=contours_r, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

    left_movement = np.sum(thresh_l)
    right_movement = np.sum(thresh_r)
        
    if left_movement >= right_movement:
        plt.imshow(frame_left)
    else:
        plt.imshow(frame_right)
        
    plt.show()
    
    #plt.imshow(thresh_l)
    #plt.show()
    #plt.imshow(right_copy)
    
    prev_left = frame_left
    prev_right = frame_right

In [ ]:
final_fps = 60.0
final_height = 1080
final_width = 1920
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')

In [ ]:
video_path = os.path.join(DATA_FOLDER, "example_keparoiCam_absolute_diff_of_cutter.avi")
video_output = cv2.VideoWriter(video_path, fourcc, final_fps, (final_width,final_height))

In [ ]:
video_left_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)
video_right_capture.set(cv2.CAP_PROP_POS_FRAMES, 0)

In [ ]:
left_n_frames = int(video_left_capture.get(cv2.CAP_PROP_FRAME_COUNT))-1
right_n_frames = int(video_right_capture.get(cv2.CAP_PROP_FRAME_COUNT))-1
total_frames = min(left_n_frames, right_n_frames)
print(total_frames)

In [ ]:
left_image = cv2.bitwise_and(left_test_image, left_test_image, mask=left_mask)
plt.imshow(left_image)
plt.show()

right_image = cv2.bitwise_and(right_test_image, right_test_image, mask=right_mask)
plt.imshow(right_image)
plt.show()

In [ ]:
# Settings
flow_fps = 5
history_length = 24

prev_left = None
prev_right = None

# Denote 0 for left frame, 1 for right frame. Always write the frame that has more "wins" in the last 10 frames
optical_flow_history = []

for i in tqdm(range(total_frames)):
    
    res_left, frame_left = video_left_capture.read()
    res_right, frame_right = video_right_capture.read()    
    
    if res_left is False or res_right is False:
        break
    
    if i == 0:
        
        masked_left = prepare_frame(frame_left, left_mask)
        masked_right = prepare_frame(frame_right, right_mask)
        
        prev_left = masked_left
        prev_right = masked_right
        continue
    
    if i < flow_fps:
        video_output.write(frame_left)
    elif i % flow_fps == 0:
        
        masked_left = prepare_frame(frame_left, left_mask)
        masked_right = prepare_frame(frame_right, right_mask)
        
        thresh_l = find_absolute_diff_frame(prev_left, masked_left)
        thresh_r = find_absolute_diff_frame(prev_right, masked_right)
        
        left_movement = np.sum(thresh_l)
        right_movement = np.sum(thresh_r)
        
        if left_movement >= right_movement:
            optical_flow_history.append(0)
        else:
            optical_flow_history.append(1)
            
        last_history = optical_flow_history[-history_length:]
        if np.mean(last_history) < 0.5:
            #print("Write left")
            video_output.write(frame_left)
        else:
            #print("Write right")
            video_output.write(frame_right)
        
        
        #contours_l, _ = cv2.findContours(image=thresh_l, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
        #contours_r, _ = cv2.findContours(image=thresh_r, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
        #left_copy = frame_left.copy()
        #right_copy = frame_right.copy()
        #cv2.drawContours(image=left_copy, contours=contours_l, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
        #cv2.drawContours(image=right_copy, contours=contours_r, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
        #plt.imshow(left_copy)
        #plt.show()
        #plt.imshow(right_copy)
        #plt.show()
            
    else:
        last_history = optical_flow_history[-history_length:]
        if np.mean(last_history) < 0.5:
            video_output.write(frame_left)
        else:
            video_output.write(frame_right)

    prev_left = masked_left
    prev_right = masked_right
    
video_left_capture.release()
video_right_capture.release()
video_output.release()